In [7]:
# import the necessary packages
import numpy as np
import imutils
import cv2
import os
import skimage
import itertools
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score
from imutils import paths

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [50]:
# grab the list of images that we'll be describing
print("[INFO] describing images...")
imagePaths = list(paths.list_images("."+os.sep+"train_vega"))

# initialize the raw pixel intensities matrix, the features matrix,
# and labels list
rawImages = []
features = []
labels = []
images=[]

[INFO] describing images...


In [51]:
#Function for the images treatment
def image_to_feature_vector(image, size=(32, 32)):
    # resize the image to a fixed size, then flatten the image into
    # a list of raw pixel intensities
    return cv2.resize(image, size).flatten()

def extract_color_histogram(image, bins=(8, 8, 8)):
    # extract a 3D color histogram from the HSV color space using
    # the supplied number of `bins` per channel
    #hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist(image, [0, 1, 2], None, bins,[0, 180, 0, 256, 0, 256])
    
    # handle normalizing the histogram if we are using OpenCV 2.4.X
    if imutils.is_cv2():
        hist = cv2.normalize(hist)

    # otherwise, perform "in place" normalization in OpenCV 3
    else:
        cv2.normalize(hist, hist)
        
    # return the flattened histogram as the feature vector
    return hist.flatten()

#Le number corresponds au nombre d'images qui vont etre générées
#Donc plus il est faible moins longs seront les calculs
def create_distortion(image,number):
    images = []
    for i in range(1,number):
        images.append(np.uint8(image + i*1* image.std() * np.random.random(image.shape)))
    return images

def image_treatment(image,label):
    rawImages.append(image_to_feature_vector(image))
    features.append(extract_color_histogram(image))
    labels.append(label)

In [52]:
# loop over the input images
for (i, imagePath) in enumerate(imagePaths):
    # load the image and extract the class label (assuming that our
    # path as the format: /path/to/dataset/{class}.{image_num}.jpg
    
    image = cv2.imread(imagePath)
    tab_image = create_distortion(image,11)
    
    image_flip = cv2.flip(image,1)
    tab_image_flip = create_distortion(image_flip,11)
   
    images = tab_image + tab_image_flip
    images.append(image)
    images.append(image_flip)
    
    label = imagePath.split(os.path.sep)[-1].split(".")[0]

    for im in images:
        image_treatment(im,label)

    # show an update every 10 images
    if i > 0 and i % 10 == 0:
        print("[INFO] processed {}/{}".format(i, len(imagePaths)))
print("Done")

[INFO] processed 10/60
[INFO] processed 20/60
[INFO] processed 30/60
[INFO] processed 40/60
[INFO] processed 50/60
Done


In [54]:
# partition the data into training and testing splits, using 75%
# of the data for training and the remaining 25% for testing
(trainRI, testRI, trainRL, testRL) = train_test_split(
    rawImages, labels, test_size=0.25, random_state=42)
(trainFeat, testFeat, trainLabels, testLabels) = train_test_split(
    features, labels, test_size=0.25, random_state=42)

In [55]:
# recherche du meilleur k 
neighbors = list(range(1,11))

# train and evaluate a k-NN classifer on the raw pixel intensities
print("[INFO] evaluating raw pixel accuracy...")
for k in neighbors :
    model = KNeighborsClassifier(n_neighbors=k, n_jobs=10)
    model.fit(trainRI, trainRL)
    acc = model.score(testRI, testRL)
    print("[INFO] raw pixel accuracy for ",k," neighbors : {:.2f}% ". format(acc * 100))
    print(metrics.classification_report(testRL, model.predict(testRI)))
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

    
print("-----------------------------------------------------------------")
print("-----------------------------------------------------------------")

# train and evaluate a k-NN classifer on the histogram
# representations
print("[INFO] evaluating histogram accuracy...")
for k in neighbors : 
    model = KNeighborsClassifier(n_neighbors=k, n_jobs=10)
    model.fit(trainFeat, trainLabels)
    acc = model.score(testFeat, testLabels)
    print("[INFO] histogram accuracy for ",k," neighbors : {:.2f}% ". format(acc * 100))
    print(metrics.classification_report(testLabels, model.predict(testFeat)))
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

[INFO] evaluating raw pixel accuracy...
[INFO] raw pixel accuracy for  1  neighbors : 10.91% 
             precision    recall  f1-score   support

       Ankh       0.33      0.01      0.02       122
     bouche       0.29      0.16      0.21        61
      hibou       0.27      0.04      0.07        92
      lapin       0.07      0.80      0.13        25
       oeil       0.33      0.03      0.06        30

avg / total       0.29      0.11      0.08       330

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[INFO] raw pixel accuracy for  2  neighbors : 13.64% 
             precision    recall  f1-score   support

       Ankh       0.56      0.04      0.08       122
     bouche       0.23      0.25      0.24        61
      hibou       0.24      0.09      0.13        92
      lapin       0.08      0.68      0.14        25
       oeil       0.00      0.00      0.00        30

avg / total       0.32      0.14      0.12       330

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

       Ankh       0.33      0.01      0.02       122
     bouche       0.24      0.21      0.22        61
      hibou       0.17      0.02      0.04        92
      lapin       0.07      0.76      0.13        25
       oeil       0.00      0.00      0.00        30

avg / total       0.22      0.11      0.07       330

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[INFO] raw pixel accuracy for  5  neighbors : 9.70% 
             precision    recall  f1-score   support

       Ankh       0.50      0.01      0.02       122
     bouche       0.18      0.16      0.17        61
      hibou       0.08      0.01      0.02        92
      lapin       0.08      0.80      0.14        25
       oeil       0.00      0.00      0.00        30

avg / total       0.25      0.10      0.05       330

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[INFO] raw pixel accuracy for  6  neighbors : 9.70% 
             precision